## The hybrid Production/Degradation model

We will be applying the hybrid model to the following spatial system:

$$
\emptyset \xrightarrow{k_1} A, \quad A \xrightarrow{k_2} \emptyset.
$$

Here we have rate of production equal to $k_1$ and rate of degradation equal to $k_2$. We implement this over both time and space, we run this on the following domain:

$$
\Omega = [0,L], \quad  t \in [0,T].
$$

We will divide the spatial domain into $M$ compartments, in which each compartment has a length of $h$, such that $h = L/M$. We can let there be a local total number of species $A_i$ to be within each compartment $\mathcal{C}_i$ for $i \in [1,M]$. We then define the spatial domain to be a sequence of these compartments in which there is a jump-rate of diffusion $d$ which controls the movement of species between compartments:


$$
A_1 \xrightleftharpoons[\ d\ ]{d} A_2 \xrightleftharpoons[\ d\ ]{d} \cdots \xrightleftharpoons[\ d\ ]{d} A_M
$$

We will use $D_i$ to denote the discrete particles modelled using the stochastic algorithm. We then can model the evolution of the concentration of the species, $c(x,t)$ over the domain. The PDE for the chemical concentration is given by the following:

$$
\frac{\partial c}{\partial t} = D\frac{\partial^2 c}{\partial x^2} + k_1 - k_2c
$$

we have that $d = \frac{D}{h^2}$. Given than the PDE grid is $k$ times more fine than the compartmental grid then we want to approximate the total mass within the compartment of the PDE we let this equal to $M_i$.

$$
\int_{\mathcal{C}_i} c(x,t) \, dx = M_i
$$

We can approximate $M_i$ to be the following:

$$
\int_{\mathcal{C}_i} c(x,t) \, dx = \sum_{j=ik}^{j=(i+1)k-1} c(x_j,t) \Delta x
$$


We can then approximate the total mass to be $A_i = D_i+ M_i$ at each compartment. 

Note we will exclude the production term in the PDE, only the stochastic regime will produce mass.

$$
\frac{\partial c}{\partial t} = D\frac{\partial^2 c}{\partial x^2} + \cancel{k_1} - k_2c
$$




We are using the Crank-Nicholson method to update the PDE solution. Given that $\bm{u}^i$ is the solution vector over the spatial grid at time $t=\Delta t i$. We have the following matrix operation:

$$
\bm{u}^{i+1}=\bm{M}\bm{u}^i
$$
Where:

$$
\bm{M}=\bm{M_1}^{-1}\bm{M_2}, \quad \bm{M}_1=\bm{I}\left(1+\frac{\Delta t}{2}k_2\right)-\nu \bm{H}
$$
$$
\bm{M}_2=\bm{I}\left(1-\frac{\Delta t}{2}k_2\right)+\nu \bm{H}
$$

Where $\nu = \frac{D\Delta t}{\Delta x^2}$, and $\bm{H}$ is the finite-difference matrix with zero-flux boundary conditions.

In [1]:
import numpy as np
from production_project import Hybrid, Stochastic

In [56]:

domain_length = 1 #Length of the domain
compartment_number = 10 #Number of compartments
PDE_multiple = 10 #How many PDE's points per cell (ie we make it a multiple times more fine)
total_time = 100 #The total time to run the system
timestep = 0.02 #The time step
threshold_conc = 100 #The threshold for the SSA to switch to the continuous regime
gamma = 2 #The rate of conversion
production_rate = 2 #The rate of production across the entire sim (this is later changed to be per cell, multiplied by h)
degredation_rate = 0.01 #The rate of degredation
diffusion_rate = (10e-1)/25 #The rate of diffusion (Scale down by L^2) Look at courant number
number_particles_per_cell = 1 #Number of particles initially per compartment
SSA_initial= np.ones((compartment_number), np.int64) * number_particles_per_cell #Initial conditions (within each cell) 

funky_initial = np.random.randint(10,size=compartment_number)

# SSA_initial = funky_initial
Model = Hybrid(domain_length, compartment_number, PDE_multiple, total_time, timestep, threshold_conc, gamma, production_rate, degredation_rate, diffusion_rate, SSA_initial)
print(Model.SSA_X)

Successfully initialized the hybrid model
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]


In [58]:
D_grid,C_grid,combined_grid = Model.run_simulation(number_of_repeats=20)

Running the simulations: 100%|██████████| 20/20 [00:24<00:00,  1.24s/it]

Simulation completed


In [60]:
Model.save_simulation_data(D_grid,C_grid,combined_grid, datadirectory='data')

Data saved successfully


To animate this please type in python animate.py x in the terminal where x is the skipping number of intervals (lower the skip, faster the animation)

In [61]:
SSA_model =  Stochastic(domain_length, compartment_number, total_time, timestep, production_rate, degredation_rate, diffusion_rate, SSA_initial) #ignore 

Successfully initialized the Stochastic model


In [62]:
SSA_grid = SSA_model.run_simulation(number_of_repeats=10)

print(SSA_grid[:,0])
print(SSA_grid)

Running the simulations: 100%|██████████| 10/10 [00:09<00:00,  1.02it/s]

Simulation completed
[13.4 14.  12.8 11.8 14.7 12.8 15.3 13.  13.7 13.2]
[[13.4  0.8  1.  ... 14.1 14.3 13.8]
 [14.   1.2  1.4 ... 13.4 13.5 14.2]
 [12.8  1.2  1.  ... 12.9 12.4 12.5]
 ...
 [13.   0.9  1.  ... 15.4 14.1 13.5]
 [13.7  0.8  1.2 ... 12.6 13.4 14.2]
 [13.2  1.1  0.8 ... 13.2 12.9 12.7]]


In [63]:
SSA_model.save_simulation_data(SSA_grid, datadirectory='data') #ignore

Data saved successfully
